In [1]:
# Libraries 

import numpy as np
import pandas as pd
import os
from datetime import datetime
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import math
import string

In [2]:
# Import data
kick_vg = pd.read_csv('ks-projects-201801.csv', encoding='latin1')
# Subset data for video games
kick_vg_18 = kick_vg[kick_vg['category'] == 'Video Games']
kick_vg_18.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
79,1000328150,Legacy of Svarog | a Unique 3D Action RPG and ...,Video Games,Games,USD,2015-10-30,50000.0,2015-08-31 06:33:31,1410.00,failed,38,US,1410.00,1410.00,50000.00
126,1000524949,Operation: Make Stuff,Video Games,Games,USD,2012-10-18,200.0,2012-09-18 04:46:06,306.72,successful,36,US,306.72,306.72,200.00
159,1000648918,"ADVENT SAGA, DIGITAL TACTICAL CARD GAME",Video Games,Games,USD,2014-07-31,70000.0,2014-07-01 16:01:41,15542.11,failed,260,US,15542.11,15542.11,70000.00
192,1000786724,Harold VS The Horde,Video Games,Games,USD,2015-01-18,14000.0,2014-12-19 07:49:11,113.00,failed,18,US,113.00,113.00,14000.00
199,1000811882,The Sword of Asumi Visual Novel,Video Games,Games,GBP,2014-11-10,2500.0,2014-10-19 20:35:23,3097.00,successful,107,GB,4983.78,4848.23,3913.65


In [3]:
#printing all summary of the kickstarter data
#this will give the dimensions of data set : (rows, columns)
print(kick_vg_18.shape)
#columns and data types
print(kick_vg_18.info())
#basic stats of columns
print(kick_vg_18.describe())

(11830, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11830 entries, 79 to 378592
Data columns (total 15 columns):
ID                  11830 non-null int64
name                11829 non-null object
category            11830 non-null object
main_category       11830 non-null object
currency            11830 non-null object
deadline            11830 non-null object
goal                11830 non-null float64
launched            11830 non-null object
pledged             11830 non-null float64
state               11830 non-null object
backers             11830 non-null int64
country             11830 non-null object
usd pledged         11829 non-null float64
usd_pledged_real    11830 non-null float64
usd_goal_real       11830 non-null float64
dtypes: float64(5), int64(2), object(8)
memory usage: 1.4+ MB
None
                 ID          goal       pledged       backers   usd pledged  \
count  1.183000e+04  1.183000e+04  1.183000e+04  11830.000000  1.182900e+04   
mean   1.074671e+09

In [4]:
#Distribution of data across state
percent_dist = round(kick_vg_18["state"].value_counts() / len(kick_vg_18["state"]) * 100,2)

print("State Percent: ")
print(percent_dist)

State Percent: 
failed        58.68
successful    20.17
canceled      19.97
live           0.73
suspended      0.45
Name: state, dtype: float64


In [5]:
#segregating the variables as categorical and constinuous
cat_vars=[ 'category', 'main_category', 'currency','country']
cont_vars=['goal', 'pledged', 'backers','usd pledged','usd_pledged_real','usd_goal_real']

In [6]:
#correlation of continuous variables with the dependent variable
kick_vg_18[cont_vars].corr()

,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
goal,1.000000,0.029378,0.023742,0.027500,0.027703,0.826637
pledged,0.029378,1.000000,0.917180,0.975863,0.991814,0.038074
backers,0.023742,0.917180,1.000000,0.927777,0.929744,0.033584
usd pledged,0.027500,0.975863,0.927777,1.000000,0.984973,0.038452
usd_pledged_real,0.027703,0.991814,0.929744,0.984973,1.000000,0.038776
usd_goal_real,0.826637,0.038074,0.033584,0.038452,0.038776,1.000000


In [7]:
#setting unique ID as index of the table
df_kick= kick_vg_18.set_index('ID')

In [8]:
# Filtering only for successful and failed projects# Filte 
kick_projects = kick_vg_18[(kick_vg_18['state'] == 'failed') | (kick_vg_18['state'] == 'successful')]
#converting 'successful' state to 1 and failed to 0
kick_projects['state'] = (kick_projects['state'] =='successful').astype(int)
print(kick_projects.shape)


(9328, 15)


C:\Users\Gabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
#creating derived metrics/ features

#converting the date columns from string to date format
#will use it to derive the duration of the project
kick_projects['launched_date'] = pd.to_datetime(kick_projects['launched'], format='%Y-%m-%d %H:%M:%S')
kick_projects['deadline_date'] = pd.to_datetime(kick_projects['deadline'], format='%Y-%m-%d %H:%M:%S')

C:\Users\Gabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Gabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
kick_projects= kick_projects.sort_values('launched_date',ascending=True)
kick_projects.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,launched_date,deadline_date
189213,1962819822,High Strangeness: A 12 bit action / adventure ...,Video Games,Games,USD,2009-08-18,1500.0,2009-05-20 22:57:54,1559.0,1,36,US,1559.0,1559.0,1500.0,2009-05-20 22:57:54,2009-08-18
327372,737331933,Send some kids to GameCamp!,Video Games,Games,USD,2009-07-03,3000.0,2009-06-02 20:33:11,3204.0,1,36,US,3204.0,3204.0,3000.0,2009-06-02 20:33:11,2009-07-03
370499,957742294,Innovating the way how children learn math,Video Games,Games,USD,2009-07-14,8000.0,2009-06-04 00:09:01,10.0,0,1,US,10.0,10.0,8000.0,2009-06-04 00:09:01,2009-07-14
231874,248639814,"2D, Side-Scrolling, 16-Bit Action Game for XBO...",Video Games,Games,USD,2009-07-14,2499.0,2009-06-05 01:30:49,35.0,0,2,US,35.0,35.0,2499.0,2009-06-05 01:30:49,2009-07-14
357534,891379703,Man Chair iPhone Application,Video Games,Games,USD,2009-08-25,10000.0,2009-06-25 19:48:43,0.0,0,0,US,0.0,0.0,10000.0,2009-06-25 19:48:43,2009-08-25


In [11]:
#creating features from the project name

#length of name
kick_projects['name_len'] = kick_projects.name.str.len()

# presence of !
kick_projects['name_exclaim'] = (kick_projects.name.str[-1] == '!').astype(int)

# presence of ?
kick_projects['name_question'] = (kick_projects.name.str[-1] == '?').astype(int)

# number of words in the name
kick_projects['name_words'] = kick_projects.name.apply(lambda x: len(str(x).split(' ')))

# if name is uppercase
kick_projects['name_is_upper'] = kick_projects.name.str.isupper().astype(float)

In [12]:
# normalizing goal by applying log
kick_projects['goal_log'] = np.log1p(kick_projects.goal)
kick_projects['Goal_10'] = kick_projects.goal.apply(lambda x: x // 10)
kick_projects['Goal_1000'] = kick_projects.goal.apply(lambda x: x // 1000)
kick_projects['Goal_100'] = kick_projects.goal.apply(lambda x: x // 100)
kick_projects['Goal_500'] = kick_projects.goal.apply(lambda x: x // 500)

In [13]:
#features from date column
kick_projects['duration']=(kick_projects['deadline_date']-kick_projects['launched_date']).dt.days
kick_projects['launched_quarter']= kick_projects['launched_date'].dt.quarter
kick_projects['launched_month']= kick_projects['launched_date'].dt.month
kick_projects['launched_year']= kick_projects['launched_date'].dt.year
kick_projects['launched_week']= kick_projects['launched_date'].dt.week

In [14]:
#additional features from goal, pledge and backers columns
kick_projects.loc[:,'goal_reached'] = kick_projects['pledged'] / kick_projects['goal'] # Pledged amount as a percentage of goal.
#The above field will be used to compute another metric
# In backers column, impute 0 with 1 to prevent undefined division.
kick_projects.loc[kick_projects['backers'] == 0, 'backers'] = 1 
kick_projects.loc[:,'pledge_per_backer'] = kick_projects['pledged'] / kick_projects['backers'] # Pledged amount per backer.

In [15]:
#will create percentile buckets for the goal amount in a category
kick_projects['goal_cat_perc'] =  kick_projects.groupby(['category'])['goal'].transform(
                     lambda x: pd.qcut(x, [0, .35, .70, 1.0], labels =[1,2,3]))

#will create percentile buckets for the duration in a category
kick_projects['duration_cat_perc'] =  kick_projects.groupby(['category'])['duration'].transform(
                     lambda x: pd.qcut(x, [0, .35, .70, 1.0], labels =False, duplicates='drop'))

In [16]:
#creating a metric to see number of competitors for a given project in a given quarter
#number of participants in a given category, that launched in the same year and quarter and in the same goal bucket
ks_particpants_qtr=kick_projects.groupby(['category','launched_year','launched_quarter','goal_cat_perc']).count()
ks_particpants_qtr=ks_particpants_qtr[['name']]
#since the above table has all group by columns created as index, converting them into columns
ks_particpants_qtr.reset_index(inplace=True)

#creating a metric to see number of competitors for a given project in a given month
#number of participants in a given category, that launched in the same year and month and in the same goal bucket
ks_particpants_mth=kick_projects.groupby(['category','launched_year','launched_month','goal_cat_perc']).count()
ks_particpants_mth=ks_particpants_mth[['name']]
#since the above table has all group by columns created as index, converting them into columns
ks_particpants_mth.reset_index(inplace=True)

#creating a metric to see number of competitors for a given project in a given week
#number of participants in a given category, that launched in the same year and week and in the same goal bucket
ks_particpants_wk=kick_projects.groupby(['category','launched_year','launched_week','goal_cat_perc']).count()
ks_particpants_wk=ks_particpants_wk[['name']]
#since the above table has all group by columns created as index, converting them into columns
ks_particpants_wk.reset_index(inplace=True)

In [17]:
#renaming columns of the derived table
colmns_qtr=['category', 'launched_year', 'launched_quarter', 'goal_cat_perc', 'participants_qtr']
ks_particpants_qtr.columns=colmns_qtr

colmns_mth=['category', 'launched_year', 'launched_month', 'goal_cat_perc', 'participants_mth']
ks_particpants_mth.columns=colmns_mth

colmns_wk=['category', 'launched_year', 'launched_week', 'goal_cat_perc', 'participants_wk']
ks_particpants_wk.columns=colmns_wk

In [18]:
#creating 2 metrics to get average pledge per backer for a category in a year according to the goal bucket it lies in and the success rate ie average pledged to goal ratio for the category and goal bucket in this year
#using pledge_per_backer (computed earlier) and averaging it by category in a launch year
ks_ppb_goal=pd.DataFrame(kick_projects.groupby(['category','launched_year','goal_cat_perc'])['pledge_per_backer','goal_reached'].mean())
#since the above table has all group by columns created as index, converting them into columns
ks_ppb_goal.reset_index(inplace=True)
#renaming column
ks_ppb_goal.columns= ['category','launched_year','goal_cat_perc','avg_ppb_goal','avg_success_rate_goal']

#creating a metric: the success rate ie average pledged to goal ratio for the category in this year
ks_ppb_duration=pd.DataFrame(kick_projects.groupby(['category','launched_year','duration_cat_perc'])['goal_reached'].mean())
#since the above table has all group by columns created as index, converting them into columns
ks_ppb_duration.reset_index(inplace=True)
#renaming column
ks_ppb_duration.columns= ['category','launched_year','duration_cat_perc','avg_success_rate_duration']

In [19]:
#merging the particpants column into the base table
kick_projects = pd.merge(kick_projects, ks_ppb_goal, on = ['category', 'launched_year','goal_cat_perc'], how = 'left')
kick_projects = pd.merge(kick_projects, ks_ppb_duration, on = ['category', 'launched_year','duration_cat_perc'], how = 'left')

In [20]:
#creating 2 metrics: mean and median goal amount
median_goal_cat=pd.DataFrame(kick_projects.groupby(['category','launched_year','duration_cat_perc'])['goal'].median())
#since the above table has all group by columns created as index, converting them into columns
median_goal_cat.reset_index(inplace=True)
#renaming column
median_goal_cat.columns= ['category','launched_year','duration_cat_perc','median_goal_year']

mean_goal_cat=pd.DataFrame(kick_projects.groupby(['category','launched_year','duration_cat_perc'])['goal'].mean())
#since the above table has all group by columns created as index, converting them into columns
mean_goal_cat.reset_index(inplace=True)
#renaming column
mean_goal_cat.columns= ['category','launched_year','duration_cat_perc','mean_goal_year']


In [21]:
#merging the particpants column into the base table
kick_projects = pd.merge(kick_projects, median_goal_cat, on = ['category', 'launched_year','duration_cat_perc'], how = 'left')
kick_projects = pd.merge(kick_projects, mean_goal_cat, on = ['category', 'launched_year','duration_cat_perc'], how = 'left')

In [22]:
# replacing all 'N,0"' values in the country column with 'NZERO' to avoid discrepancies while one hot encoding
kick_projects = kick_projects.replace({'country': 'N,0"'}, {'country': 'NZERO'}, regex=True)

In [23]:
#creating a list of columns to be dropped
drop_columns= ['name','launched','deadline','launched_date','deadline_date','pledged','backers','usd pledged','usd_pledged_real','pledge_per_backer','goal_reached']
#dropping columns above
kick_projects.drop(drop_columns, axis=1, inplace=True)

In [24]:
#these functions will be used on the textual column entries to remove '&','-' or white spaces
def replace_ampersand(val):
    if isinstance(val, str):
        return(val.replace('&', 'and'))
    else:
        return(val)

def replace_hyphen(val):
    if isinstance(val, str):
        return(val.replace('-', '_'))
    else:
        return(val)    
    
def remove_extraspace(val):
        if isinstance(val, str):
            return(val.strip())
        else:
            return(val) 

def replace_space(val):
        if isinstance(val, str):
            return(val.replace(' ', '_'))
        else:
            return(val)         

In [25]:
#apply those functions to all cat columns
kick_projects['category'] = kick_projects['category'].apply(remove_extraspace)
kick_projects['category'] = kick_projects['category'].apply(replace_ampersand)
kick_projects['category'] = kick_projects['category'].apply(replace_hyphen)
kick_projects['category'] = kick_projects['category'].apply(replace_space)

kick_projects['main_category'] = kick_projects['main_category'].apply(remove_extraspace)
kick_projects['main_category'] = kick_projects['main_category'].apply(replace_ampersand)
kick_projects['main_category'] = kick_projects['main_category'].apply(replace_hyphen)
kick_projects['main_category'] = kick_projects['main_category'].apply(replace_space)

In [26]:
#dropping all rows that have any nulls
kick_projects=kick_projects.dropna() 

In [27]:
#creating a backup copy of the dataset
kick_projects_copy= kick_projects.copy()

kick_projects_copy[:5]

,ID,category,main_category,currency,goal,state,country,usd_goal_real,name_len,name_exclaim,...,launched_month,launched_year,launched_week,goal_cat_perc,duration_cat_perc,avg_ppb_goal,avg_success_rate_goal,avg_success_rate_duration,median_goal_year,mean_goal_year
0,1962819822,Video_Games,Games,USD,1500.0,1,US,1500.0,57.0,0,...,5,2009,21,1.0,2,32.578874,1.617137,1.328282,1500.0,5964.6456
1,737331933,Video_Games,Games,USD,3000.0,1,US,3000.0,27.0,1,...,6,2009,23,1.0,1,32.578874,1.617137,1.068000,3000.0,3000.0000
2,957742294,Video_Games,Games,USD,8000.0,0,US,8000.0,42.0,0,...,6,2009,23,2.0,2,75.015796,0.346682,1.328282,1500.0,5964.6456
3,248639814,Video_Games,Games,USD,2499.0,0,US,2499.0,59.0,1,...,6,2009,23,1.0,2,32.578874,1.617137,1.328282,1500.0,5964.6456
4,891379703,Video_Games,Games,USD,10000.0,0,US,10000.0,28.0,0,...,6,2009,26,2.0,2,75.015796,0.346682,1.328282,1500.0,5964.6456


In [28]:
for c in kick_projects.columns:
    col_type = kick_projects[c].dtype
    if col_type == 'object' :
        a=kick_projects[c].unique()
        keys= range(a.shape[0])
        diction={}
        for idx,val in enumerate(a):
            diction[idx] = a[idx]
        diction = {v: k for k, v in diction.items()}
        print(diction)
        kick_projects_copy[c] = [diction[item] for item in kick_projects_copy[c]] 
        kick_projects_copy[c] = kick_projects_copy[c].astype('category')

{'Video_Games': 0}
{'Games': 0}
{'USD': 0, 'GBP': 1, 'CAD': 2, 'AUD': 3, 'NZD': 4, 'EUR': 5, 'SEK': 6, 'DKK': 7, 'NOK': 8, 'CHF': 9, 'HKD': 10, 'SGD': 11, 'MXN': 12, 'JPY': 13}
{'US': 0, 'GB': 1, 'CA': 2, 'AU': 3, 'NZ': 4, 'NL': 5, 'SE': 6, 'DK': 7, 'IE': 8, 'NO': 9, 'DE': 10, 'FR': 11, 'ES': 12, 'BE': 13, 'IT': 14, 'CH': 15, 'AT': 16, 'HK': 17, 'SG': 18, 'MX': 19, 'JP': 20}


In [29]:
# One-Hot encoding to convert categorical columns to numeric
print('start one-hot encoding')

kick_projects_ip = pd.get_dummies(kick_projects, prefix = [ 'category', 'main_category', 'currency','country'],
                             columns = [ 'category', 'main_category', 'currency','country'])
    
#this will have created 1-0 flag columns (like a sparse matrix)    
print('ADS dummy columns made')

start one-hot encoding
ADS dummy columns made


In [30]:
#creating 2 arrays: features and response

#features will have all independent variables
features=list(kick_projects_ip)
features.remove('state')
#response has the target variable
response= ['state']

In [31]:
#creating a backup copy of the input dataset
kick_projects_ip_copy= kick_projects_ip.copy()
kick_projects_ip[features].shape

(9327, 62)

In [32]:
# normalize the data attributes
kick_projects_ip_scaled_ftrs = pd.DataFrame(preprocessing.normalize(kick_projects_ip[features]))
kick_projects_ip_scaled_ftrs.columns=list(kick_projects_ip[features])

In [33]:
kick_projects_ip_scaled_ftrs[:3]

,ID,goal,usd_goal_real,name_len,name_exclaim,name_question,name_words,name_is_upper,goal_log,Goal_10,...,country_IE,country_IT,country_JP,country_MX,country_NL,country_NO,country_NZ,country_SE,country_SG,country_US
0,1.0,7.642067e-07,7.642067e-07,2.903985e-08,0.000000e+00,0.0,5.094711e-09,0.0,3.726214e-09,7.642067e-08,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.094711e-10
1,1.0,4.068724e-06,4.068724e-06,3.661851e-08,1.356241e-09,0.0,6.781206e-09,0.0,1.085902e-08,4.068724e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.356241e-09
2,1.0,8.352978e-06,8.352978e-06,4.385313e-08,0.000000e+00,0.0,7.308855e-09,0.0,9.383862e-09,8.352978e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.044122e-09


In [34]:
#creating test and train dependent and independent variables
train_ind, test_ind, train_dep, test_dep = train_test_split(kick_projects_ip_scaled_ftrs, kick_projects_ip[response], test_size=0.3, random_state=0)

In [36]:
features_count = train_ind.shape[1]

parameters_rf = {'n_estimators':[50], 'max_depth':[20], 'max_features': 
                     [math.floor(np.sqrt(features_count)), math.floor(features_count/3)]}

def random_forest_classifier(features, target):
    clf = RandomForestClassifier(n_estimators=50,criterion='gini' ,max_depth=20, max_features=2)
    clf.fit(features, target)
    return clf

In [37]:
trained_model_RF= random_forest_classifier(train_ind[features], train_dep[response])

C:\Users\Gabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [38]:
# Predict the on the train_data
test_ind["Pred_state_RF"] = trained_model_RF.predict(test_ind[features])

# Predict the on the train_data
train_ind["Pred_state_RF"] = trained_model_RF.predict(train_ind[features])

# Predict the on the train_data
kick_projects_ip["Pred_state_RF"] = trained_model_RF.predict(kick_projects_ip_scaled_ftrs)

C:\Users\Gabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Gabe\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [44]:
# Train and Test Accuracy
print ("Train Accuracy :: ", accuracy_score(train_dep[response], trained_model_RF.predict(train_ind[features])))
print ("Test Accuracy  :: ", accuracy_score(test_dep[response], trained_model_RF.predict(test_ind[features])))
print ("Complete Accuracy  :: ", accuracy_score(kick_projects_ip[response], trained_model_RF.predict(kick_projects_ip_scaled_ftrs)))


Train Accuracy ::  0.7469362745098039
Test Accuracy  ::  0.7431225437656306
Complete Accuracy  ::  0.7457917872842286


In [40]:
## Feature importances
ftr_imp_rf=zip(features,trained_model_RF.feature_importances_)
for values in ftr_imp_rf:
    print(values)

('ID', 0.03590079648525926)
('goal', 0.11860696207831369)
('usd_goal_real', 0.10473648973708663)
('name_len', 0.0436839729555594)
('name_exclaim', 0.001149386816377754)
('name_question', 0.0)
('name_words', 0.01720350721346856)
('name_is_upper', 0.0)
('goal_log', 0.044380717683694094)
('Goal_10', 0.06665529044016055)
('Goal_1000', 0.02522862228499891)
('Goal_100', 0.032716935680391065)
('Goal_500', 0.03408567164781)
('duration', 0.03614385873756798)
('launched_quarter', 0.007161873278295769)
('launched_month', 0.023697851283049975)
('launched_year', 0.058582487209857544)
('launched_week', 0.045129247800000695)
('goal_cat_perc', 0.014171904034072692)
('duration_cat_perc', 0.0023791085392616002)
('avg_ppb_goal', 0.036070948134927394)
('avg_success_rate_goal', 0.030768389048386562)
('avg_success_rate_duration', 0.02054485916990908)
('median_goal_year', 0.0597693846864976)
('mean_goal_year', 0.08810493898898916)
('category_Video_Games', 0.003321266801286519)
('main_category_Games', 0.00487

In [41]:
feature_imp_RF=pd.DataFrame(list(zip(features,trained_model_RF.feature_importances_)))
column_names_RF= ['features','RF_imp']
feature_imp_RF.columns= column_names_RF

In [43]:
feature_imp_RF= feature_imp_RF.sort_values('RF_imp',ascending=False)
feature_imp_RF[:10]

,features,RF_imp
1,goal,0.118607
2,usd_goal_real,0.104736
24,mean_goal_year,0.088105
9,Goal_10,0.066655
23,median_goal_year,0.059769
16,launched_year,0.058582
17,launched_week,0.045129
8,goal_log,0.044381
3,name_len,0.043684
13,duration,0.036144
